In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import Sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Function to add Gaussian noise
def add_noise(img):
    noise_factor = 0.1  # Adjust for noise level
    noisy_img = img + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    noisy_img = np.clip(noisy_img, 0.0, 1.0)  # Ensure values are between 0 and 1
    return noisy_img

# Custom Data Generator with noise addition
class NoisyImageDataGenerator(Sequence):
    def __init__(self, generator):
        self.generator = generator

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        noisy_images = np.array([add_noise(img) for img in images])  # Apply noise
        return noisy_images, labels

In [ ]:
# Setting up paths
train_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Train'
val_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Validation'
test_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Test'

In [ ]:
# Data generators with augmentation for training
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Xception input size
    batch_size=32,
    class_mode='binary'
)
train_generator = NoisyImageDataGenerator(train_generator)  # Add noise

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


In [ ]:
# Load the Xception model without the top layer (classifier)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build the model with additional layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator
)

In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Predictions and evaluation metrics
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cbar=False, linewidths=1,
            linecolor='black', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:")
print(report)

In [ ]:
# Plot Performance Metrics
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Check Test Data Predictions (Real or Fake)
test_filenames = test_generator.filenames

In [ ]:
# Define the number of images to display
num_images_to_display = 20

In [ ]:
# Create a grid layout for better visualization
rows, cols = 4, 5
plt.figure(figsize=(15, 12))

for i in range(num_images_to_display):
    img_path = test_dir + '/' + test_filenames[i]
    img = plt.imread(img_path)

    # Get the prediction
    prediction = 'Real' if predicted_classes[i][0] == 0 else 'Fake'

    # Plot the image
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.title(f"Predicted: {prediction}", fontsize=10)
    plt.axis('off')

# Add a title to the overall grid
plt.suptitle("Test Data Predictions (Real or Fake)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
